In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import numpy as np
from os import listdir
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cv2
import torchvision.transforms as transforms
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [34]:
directory_path = '/kaggle/input/4d-ai-lab/processed_data/'
file_types = ['npy']
np_vars = []
for dir_content in listdir(directory_path): 
    if dir_content.split('.')[-1] in file_types:
        np_vars.append(np.load(directory_path+dir_content).T)
        
np_vars = np.array(np_vars,dtype='float32')
np_vars = np_vars/255.0

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
def rotate(arr):
    
    a1 = np.rot90(arr,axes=(1,2)) ### 90
    a2 =  np.rot90(a1,axes=(1,2)) ### 180
    a3 = np.rot90(a2,axes=(1,2)) ### 270
    
    
    return arr, a1, a2, a3, 0, 1, 2,3

In [5]:
t = []
for i in np_vars:
    t.append(i.T)
t = np.array(t)
np_vars = None
t.shape

(4910, 3, 256, 256)

In [6]:
X=None
Y=None
X=[]
Y=[]
for i in t:
    a1,a2,a3,a4,y1,y2,y3,y4 = rotate(i)
    X.append(a1)
    X.append(a2)
    X.append(a3)
    X.append(a4)
    
    Y.append(y1)
    Y.append(y2)
    Y.append(y3)
    Y.append(y4)

t = None
print(X[0].shape)
X = np.array(X)
Y = np.array(Y)
print(X.shape,Y.shape)

(3, 256, 256)
(19640, 3, 256, 256) (19640,)


In [8]:
print(X.shape)

(19640, 3, 256, 256)


In [7]:
i=None
a1= None
a2=None
a3=None
a4 = None
y1 = None
y2 = None
y3 = None
y4 = None

In [8]:
X_torch = torch.from_numpy(X).to(device)
Y_torch = torch.from_numpy(Y).to(device)
X=None
Y=None

In [9]:
class RotationPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(32 * 64 * 64, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 4)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x)
        x = self.classifier(x)
        return x.softmax(dim=0)


In [10]:
model = RotationPredictor().to(device)
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
epochs=10
for epoch in range(epochs):
    running_loss = 0.0
    for i in range(len(X_torch)):
        optimizer.zero_grad()
        
        outputs = model(X_torch[i])
        Y_torch[i]=Y_torch[i].type(torch.cuda.FloatTensor)
        #outputs = outputs.type(torch.cuda.LongTensor)
        #print(outputs.unsqueeze(0), Y_torch[i].unsqueeze(0).unsqueeze(1))
        loss = criterion(outputs, Y_torch[i].long())
        loss.backward()
        
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / 19460
    print(f"Epoch {epochs+1}/{epochs}, Loss: {epoch_loss:.4f}")

Epoch 11/10, Loss: 1.5075
Epoch 11/10, Loss: 1.5075
Epoch 11/10, Loss: 1.5075
Epoch 11/10, Loss: 1.5075
Epoch 11/10, Loss: 1.5075
Epoch 11/10, Loss: 1.5075
Epoch 11/10, Loss: 1.5075
Epoch 11/10, Loss: 1.5075
Epoch 11/10, Loss: 1.5075
Epoch 11/10, Loss: 1.5075


In [39]:
t = []

for i in range(len(X_torch)):
    
    if i%4==0:
        r = model.features(X_torch[i]).detach().cpu().numpy()
        t.append(r.flatten())
    

In [40]:
t = np.array(t)
print(t.shape)

(4910, 131072)


In [45]:
kmeans = KMeans(n_clusters=2, random_state=20, n_init="auto").fit(t)

count = 0
HashList = []
for dir_content in listdir(directory_path): 
    HashList.append({'id': f'{count}','labels':kmeans.labels_[count]})
    count+=1
    
df = pd.DataFrame(HashList)
df.to_csv('my_file.csv', index=False, header=True)

In [30]:
count = 0
HashList = []
for dir_content in listdir(directory_path): 
    HashList.append({'id': f'{count}','labels':kmeans.labels_[count]})
    count+=1

In [31]:
df = pd.DataFrame(HashList)
df.to_csv('my_file.csv', index=False, header=True)